In [1]:
require 'Loader'
require 'Mapper'

In [2]:
indexer_1 = indexer('./prepare_datasets/an4_lmdb/train/', 20)

In [3]:
mapper = Mapper('./dictionary')
loader = Loader('./prepare_datasets/an4_lmdb/train/', mapper)

In [4]:
inds = indexer_1:nextIndices()

In [5]:
batch_data = loader:nextBatch(inds)

In [6]:
batch_data:size()

  20
   1
 161
  79
[torch.LongStorage of size 4]



In [7]:
Network = require 'Network'

In [8]:
cmd = torch.CmdLine()
cmd:option('-loadModel', false, 'Load previously saved model')
cmd:option('-saveModel', true, 'Save model after training/testing')
cmd:option('-modelName', 'DeepSpeechModel', 'Name of class containing architecture')
cmd:option('-nGPU', 1, 'Number of GPUs, set -1 to use CPU')
cmd:option('-trainingSetLMDBPath', './prepare_datasets/an4_lmdb/train/', 'Path to LMDB training dataset')
cmd:option('-validationSetLMDBPath', './prepare_datasets/an4_lmdb/test/', 'Path to LMDB test dataset')
cmd:option('-logsTrainPath', './logs/TrainingLoss/', ' Path to save Training logs')
cmd:option('-logsValidationPath', './logs/ValidationScores/', ' Path to save Validation logs')
cmd:option('-saveModelInTraining', false, 'save model periodically through training')
cmd:option('-modelTrainingPath', './models/', ' Path to save periodic training models')
cmd:option('-saveModelIterations', 50, 'When to save model through training')
cmd:option('-modelPath', 'deepspeech.t7', 'Path of final model to save/load')
cmd:option('-dictionaryPath', './dictionary', ' File containing the dictionary to use')
cmd:option('-epochs', 70, 'Number of epochs for training')
cmd:option('-learningRate', 3e-4, ' Training learning rate')
cmd:option('-learningRateAnnealing', 1.1, 'Factor to anneal lr every epoch')
cmd:option('-maxNorm', 400, 'Max norm used to normalize gradients')
cmd:option('-momentum', 0.90, 'Momentum for SGD')
cmd:option('-batchSize', 20, 'Batch size in training')
cmd:option('-validationBatchSize', 20, 'Batch size for validation')
cmd:option('-hiddenSize', 1760, 'RNN hidden sizes')
cmd:option('-nbOfHiddenLayers', 7, 'Number of rnn layers')
opt = cmd:parse("")

In [9]:
optimParams = {
    learningRate = opt.learningRate,
    learningRateAnnealing = opt.learningRateAnnealing,
    momentum = opt.momentum,
    dampening = 0,
    nesterov = true
}

In [10]:
Network:init(opt)

In [11]:
inputs = batch_data:cuda()
print(inputs:size())

  20
   1
 161
  79
[torch.LongStorage of size 4]



In [12]:
Network.model:get(1)

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> output]
  (1): cudnn.SpatialConvolution(1 -> 32, 11x41, 2,2)
  (2): nn.SpatialBatchNormalization
  (3): nn.Clamp
  (4): cudnn.SpatialConvolution(32 -> 32, 11x21, 2,1)
  (5): nn.SpatialBatchNormalization
  (6): nn.Clamp
  (7): nn.View(1312, -1)
  (8): nn.Transpose
}
{
  gradInput : CudaTensor - empty
  modules : 
    {
      1 : 
        cudnn.SpatialConvolution(1 -> 32, 11x41, 2,2)
        {
          dH : 2
          dW : 2
          nInputPlane : 1
          output : CudaTensor - empty
          kH : 41
          bias : CudaTensor - size: 32
          padH : 0
          gradBias : CudaTensor - size: 32
          weight : CudaTensor - size: 32x1x41x11
          _type : torch.CudaTensor
          gradWeight : CudaTensor - size: 32x1x41x11
          padW : 0
          nOutputPlane : 32
          kW : 11
          gradInput : CudaTensor - empty
        }
      2 : 
        nn.SpatialBatchNormalization
   

          kW : 11
          gradInput : CudaTensor - empty
        }
      5 : 
        nn.SpatialBatchNormalization
        {
          gradBias : CudaTensor - size: 32
          output : CudaTensor - empty
          gradInput : CudaTensor - empty
          running_var : CudaTensor - size: 32
          momentum : 0.1
          gradWeight : CudaTensor - size: 32
          eps : 1e-05
          _type : torch.CudaTensor
          affine : true
          running_mean : CudaTensor - size: 32
          bias : CudaTensor - size: 32
          weight : CudaTensor - size: 32
          train : true
        }
      6 : 
        nn.Clamp
        {
          inplace : false
          min_val : 0
          output : CudaTensor - empty
          gradInput : CudaTensor - empty
          max_val : 20
          _type : torch.CudaTensor
        }
      7 : 
        nn.View(1312, -1)
        {
          _type : torch.CudaTensor
          output : CudaTensor - empty
          gradInput : CudaTensor - empty


In [13]:
d1 = Network.model:get(1):forward(inputs)

In [16]:
d1:size()

   13
   20
 1312
[torch.LongStorage of size 3]



In [57]:
Network.model:get(2)

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> output]
  (1): BatchBRNN {
    [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> output]
    (1): nn.View(260, -1)
    (2): nn.Linear(1312 -> 1760) without bias
    (3): nn.BatchNormalization
    (4): nn.View(13, 20, -1)
    (5): cudnn.RNN
    (6): nn.Clamp
    (7): nn.View(-1, 2, 1760)
    (8): nn.Sum
  }
  (2): nn.Bottle
  (3): BatchBRNN {
    [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> output]
    (1): nn.View(260, -1)
    (2): nn.Linear(1760 -> 1760) without bias
    (3): nn.BatchNormalization
    (4): nn.View(13, 20, -1)
    (5): cudnn.RNN
    (6): nn.Clamp
    (7): nn.View(-1, 2, 1760)
    (8): nn.Sum
  }
  (4): nn.Bottle
  (5): BatchBRNN {
    [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> output]
    (1): nn.View(260, -1)
    (2): nn.Linear(1760 -> 1760) without bias
    (3): nn.BatchNormalization
   

     _type : torch.CudaTensor
              output : CudaTensor - size: 260x1312
              gradInput : CudaTensor - empty
              size : LongStorage - size: 2
              numElements : 260
              numInputDims : 3
            }
          _type : torch.CudaTensor
          output : CudaTensor - size: 13x20x1760
          gradInput : CudaTensor - empty
          modules : 
            {
              1 : 
                nn.View(260, -1)
                {
                  _type : torch.CudaTensor
                  output : CudaTensor - size: 260x1312
                  gradInput : CudaTensor - empty
                  size : LongStorage - size: 2
                  numElements : 260
                  numInputDims : 3
                }
              2 : 
                nn.Linear(1312 -> 1760) without bias
                {
                  weight : CudaTensor - size: 1760x1312
                  _type : torch.CudaTensor
                  output : CudaTensor - size: 260x17

               addBuffer : CudaTensor - size: 260
                  gradWeight : CudaTensor - size: 1760x1312
                }
              3 : 
                nn.BatchNormalization
                {
                  gradBias : CudaTensor - size: 1760
                  bias : CudaTensor - size: 1760
                  output : CudaTensor - size: 260x1760
                  gradInput : CudaTensor - empty
                  save_mean : CudaTensor - size: 1760
                  running_var : CudaTensor - size: 1760
                  momentum : 0.1
                  _type : torch.CudaTensor
                  eps : 1e-05
                  train : true
                  affine : true
                  running_mean : CudaTensor - size: 1760
                  weight : CudaTensor - size: 1760
                  save_std : CudaTensor - size: 1760
                  gradWeight : CudaTensor - size: 1760
                }
              4 : 
                nn.View(13, 20, -1)
                {
     

          _type : torch.CudaTensor
                  output : CudaTensor - size: 13x20x1760
                  gradInput : CudaTensor - empty
                  size : LongStorage - size: 3
                  numElements : 260
                  numInputDims : 2
                }
              5 : 
                cudnn.RNN
                {
                  yDescs : cdata<struct cudnnTensorStruct *[?]>: 0x4081f6b0
                  sync : true
                  rememberStates : false
                  output : CudaTensor - size: 13x20x3520
                  reserve : CudaTensor - size: 915200
                  batchFirst : false
                  hxDesc : cdata<struct cudnnTensorStruct *[?]>: 0x41057dc0
                  hiddenSize : 1760
                  gradHiddenInput : CudaTensor - empty
                  cellOutput : CudaTensor - size: 2x20x1760
                  rnnDesc : cdata<struct cudnnRNNStruct *[?]>: 0x41a496c8
                  gradWeight : CudaTensor - size: 12397441
     

seed : 19088743
                  cxDesc : cdata<struct cudnnTensorStruct *[?]>: 0x40d88270
                  gradInput : CudaTensor - empty
                  _type : torch.CudaTensor
                  hyDesc : cdata<struct cudnnTensorStruct *[?]>: 0x41057f10
                  wDesc : cdata<struct cudnnFilterStruct *[?]>: 0x403d6708
                  seqLength : 13
                  cyDesc : cdata<struct cudnnTensorStruct *[?]>: 0x40d89e28
                  bidirectional : CUDNN_BIDIRECTIONAL
                  numDirections : 2
                  gradCellInput : CudaTensor - empty
                  miniBatch : 20
                  mode : CUDNN_RNN_RELU
                  dropoutStatesSize : LongTensor - size: 1
                  hiddenOutput : CudaTensor - size: 2x20x1760
                  datatype : CUDNN_DATA_FLOAT
                  weight : CudaTensor - size: 12397441
                  train : true
                  numLayers : 1
                  inputSize : 1760
                  in

...

          output : CudaTensor - size: 13x20x1760
          gradInput : CudaTensor - empty
          modules : 
            {
              1 : 
                nn.BatchNormalization
                {
                  gradBias : CudaTensor - size: 1760
                  weight : CudaTensor - size: 1760
                  output : CudaTensor - size: 260x1760
                  gradInput : CudaTensor - empty
                  save_mean : CudaTensor - size: 1760
                  running_var : CudaTensor - size: 1760
                  momentum : 0.1
                  gradWeight : CudaTensor - size: 1760
                  eps : 1e-05
                  _type : torch.CudaTensor
                  affine : true
                  running_mean : CudaTensor - size: 1760
                  bias : CudaTensor - size: 1760
                  save_std : CudaTensor - size: 1760
                  train : true
                }
            }
          dimDelta : 0
          _type : torch.CudaTensor
         

       nn.View(260, -1)
                {
                  _type : torch.CudaTensor
                  output : CudaTensor - size: 260x1760
                  gradInput : CudaTensor - empty
                  size : LongStorage - size: 2
                  numElements : 260
                  numInputDims : 3
                }
              2 : 
                nn.Linear(1760 -> 1760) without bias
                {
                  weight : CudaTensor - size: 1760x1760
                  _type : torch.CudaTensor
                  output : CudaTensor - size: 260x1760
                  gradInput : CudaTensor - empty
                  addBuffer : CudaTensor - size: 260
                  gradWeight : CudaTensor - size: 1760x1760
                }
              3 : 
                nn.BatchNormalization
                {
                  gradBias : CudaTensor - size: 1760
                  bias : CudaTensor - size: 1760
                  output : CudaTensor - size: 260x1760
                  g

                  size : LongStorage - size: 3
                  numElements : 260
                  numInputDims : 2
                }
              5 : 
                cudnn.RNN
                {
                  yDescs : cdata<struct cudnnTensorStruct *[?]>: 0x407b8df8
                  sync : true
                  rememberStates : false
                  output : CudaTensor - size: 13x20x3520
                  reserve : CudaTensor - size: 915200
                  batchFirst : false
                  hxDesc : cdata<struct cudnnTensorStruct *[?]>: 0x40a95638
                  hiddenSize : 1760
                  gradHiddenInput : CudaTensor - empty
                  cellOutput : CudaTensor - size: 2x20x1760
                  rnnDesc : cdata<struct cudnnRNNStruct *[?]>: 0x41433548
                  gradWeight : CudaTensor - size: 12397441
                  dropout : 0
                  gradCellInput : CudaTensor - empty
                  inputMode : CUDNN_SKIP_INPUT
                

                xDescs : cdata<struct cudnnTensorStruct *[?]>: 0x4066d320
                }
              6 : 
                nn.Clamp
                {
                  inplace : false
                  min_val : 0
                  output : CudaTensor - size: 13x20x3520
                  gradInput : CudaTensor - empty
                  max_val : 20
                  _type : torch.CudaTensor
                }
              7 : 
                nn.View(-1, 2, 1760)
                {
                  _type : torch.CudaTensor
                  output : CudaTensor - size: 13x20x2x1760
                  gradInput : CudaTensor - empty
                  size : LongStorage - size: 3
                  numElements : 3520
                  numInputDims : 2
                }
              8 : 
                nn.Sum
                {
                  _type : torch.CudaTensor
                  output : CudaTensor - size: 13x20x1760
                  gradInput : CudaTensor - empty
             

            cellOutput : CudaTensor - size: 2x20x1760
              rnnDesc : cdata<struct cudnnRNNStruct *[?]>: 0x41433548
              gradWeight : CudaTensor - size: 12397441
              dropout : 0
              gradCellInput : CudaTensor - empty
              inputMode : CUDNN_SKIP_INPUT
              cxDesc : cdata<struct cudnnTensorStruct *[?]>: 0x418f9ab8
              gradInput : CudaTensor - empty
              _type : torch.CudaTensor
              hyDesc : cdata<struct cudnnTensorStruct *[?]>: 0x40b63590
              wDesc : cdata<struct cudnnFilterStruct *[?]>: 0x41fba808
              seqLength : 13
              cyDesc : cdata<struct cudnnTensorStruct *[?]>: 0x406c0de8
              bidirectional : CUDNN_BIDIRECTIONAL
              dropoutStates : CudaTensor - size: 196608
              inputSize : 1760
              dropoutStatesSize : LongTensor - size: 1
              miniBatch : 20
              hiddenOutput : CudaTensor - size: 2x20x1760
              mode : CUD

In [62]:
Network.model:get(2):get(1)

BatchBRNN {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> output]
  (1): nn.View(260, -1)
  (2): nn.Linear(1312 -> 1760) without bias
  (3): nn.BatchNormalization
  (4): nn.View(13, 20, -1)
  (5): cudnn.RNN
  (6): nn.Clamp
  (7): nn.View(-1, 2, 1760)
  (8): nn.Sum
}
{
  view_in : 
    nn.View(260, -1)
    {
      _type : torch.CudaTensor
      output : CudaTensor - size: 260x1312
      gradInput : CudaTensor - empty
      size : LongStorage - size: 2
      numElements : 260
      numInputDims : 3
    }
  _type : torch.CudaTensor
  output : CudaTensor - size: 13x20x1760
  gradInput : CudaTensor - empty
  modules : 
    {
      1 : 
        nn.View(260, -1)
        {
          _type : torch.CudaTensor
          output : CudaTensor - size: 260x1312
          gradInput : CudaTensor - empty
          size : LongStorage - size: 2
          numElements : 260
          numInputDims : 3
        }
      2 : 
        nn.Linear(1312 -> 1760) without bias
        {
          w

          weight : CudaTensor - size: 1760
          save_std : CudaTensor - size: 1760
          gradWeight : CudaTensor - size: 1760
        }
      4 : 
        nn.View(13, 20, -1)
        {
          _type : torch.CudaTensor
          output : CudaTensor - size: 13x20x1760
          gradInput : CudaTensor - empty
          size : LongStorage - size: 3
          numElements : 260
          numInputDims : 2
        }
      5 : 
        cudnn.RNN
        {
          yDescs : cdata<struct cudnnTensorStruct *[?]>: 0x4081f6b0
          sync : true
          rememberStates : false
          output : CudaTensor - size: 13x20x3520
          reserve : CudaTensor - size: 915200
          batchFirst : false
          hxDesc : cdata<struct cudnnTensorStruct *[?]>: 0x41057dc0
          hiddenSize : 1760
          gradHiddenInput : CudaTensor - empty
          cellOutput : CudaTensor - size: 2x20x1760
          rnnDesc : cdata<struct cudnnRNNStruct *[?]>: 0x41a496c8
          gradWeight : CudaTen

       _type : torch.CudaTensor
          output : CudaTensor - size: 13x20x2x1760
          gradInput : CudaTensor - empty
          size : LongStorage - size: 3
          numElements : 3520
          numInputDims : 2
        }
      8 : 
        nn.Sum
        {
          _type : torch.CudaTensor
          output : CudaTensor - size: 13x20x1760
          gradInput : CudaTensor - empty
          dimension : 3
          sizeAverage : false
        }
    }
  rnn : 
    cudnn.RNN
    {
      yDescs : cdata<struct cudnnTensorStruct *[?]>: 0x4081f6b0
      sync : true
      rememberStates : false
      output : CudaTensor - size: 13x20x3520
      reserve : CudaTensor - size: 915200
      batchFirst : false
      hxDesc : cdata<struct cudnnTensorStruct *[?]>: 0x41057dc0
      hiddenSize : 1760
      gradHiddenInput : CudaTensor - empty
      cellOutput : CudaTensor - size: 2x20x1760
      rnnDesc : cdata<struct cudnnRNNStruct *[?]>: 0x41a496c8
      gradWeight : CudaTensor - size: 12397441


In [84]:
print(d1:size())

   13
   20
 1312
[torch.LongStorage of size 3]



In [85]:
print(Network.model:get(2):get(1):get(1))
d11 = Network.model:get(2):get(1):get(1):forward(d1)
print(d11:size())

nn.View(260, -1)
{
  _type : torch.CudaTensor
  output : CudaTensor - size: 260x1312
  gradInput : CudaTensor - empty
  size : LongStorage - size: 2
  numElements : 260
  numInputDims : 3
}
  260
 1312
[torch.LongStorage of size 2]



In [86]:
print(Network.model:get(2):get(1):get(2))
d12 = Network.model:get(2):get(1):get(2):forward(d11)
print(d12:size())

nn.Linear(1312 -> 1760) without bias
{
  weight : CudaTensor - size: 1760x1312
  _type : torch.CudaTensor
  output : CudaTensor - size: 260x1760
  gradInput : CudaTensor - empty
  addBuffer : CudaTensor - size: 260
  gradWeight : CudaTensor - size: 1760x1312
}
  260
 1760
[torch.LongStorage of size 2]



In [87]:
print(Network.model:get(2):get(1):get(3))
d13 = Network.model:get(2):get(1):get(3):forward(d12)
print(d13:size())

nn.BatchNormalization
{
  gradBias : CudaTensor - size: 1760
  bias : CudaTensor - size: 1760
  output : CudaTensor - size: 260x1760
  gradInput : CudaTensor - empty
  save_mean : CudaTensor - size: 1760
  running_var : CudaTensor - size: 1760
  momentum : 0.1
  _type : torch.CudaTensor
  eps : 1e-05
  train : true
  affine : true
  running_mean : CudaTensor - size: 1760
  weight : CudaTensor - size: 1760
  save_std : CudaTensor - size: 1760
  gradWeight : CudaTensor - size: 1760
}
  260
 1760
[torch.LongStorage of size 2]



In [88]:
print(Network.model:get(2):get(1):get(4))
d14 = Network.model:get(2):get(1):get(4):forward(d13)
print(d14:size())

nn.View(13, 20, -1)
{
  _type : torch.CudaTensor
  output : CudaTensor - size: 13x20x1760
  gradInput : CudaTensor - empty
  size : LongStorage - size: 3
  numElements : 260
  numInputDims : 2
}
   13
   20
 1760
[torch.LongStorage of size 3]



In [90]:
print(Network.model:get(2):get(1):get(5))
d15 = Network.model:get(2):get(1):get(5):forward(d14)
print(d15:size())

cudnn.RNN
{
  yDescs : cdata<struct cudnnTensorStruct *[?]>: 0x4081f6b0
  sync : true
  rememberStates : false
  output : CudaTensor - size: 13x20x3520
  reserve : CudaTensor - size: 915200
  batchFirst : false
  hxDesc : cdata<struct cudnnTensorStruct *[?]>: 0x41057dc0
  hiddenSize : 1760
  gradHiddenInput : CudaTensor - empty
  cellOutput : CudaTensor - size: 2x20x1760
  rnnDesc : cdata<struct cudnnRNNStruct *[?]>: 0x41a496c8
  gradWeight : CudaTensor - size: 12397441
  dropout : 0
  dropoutStates : CudaTensor - size: 196608
  seed : 19088743
  cxDesc : cdata<struct cudnnTensorStruct *[?]>: 0x40d88270
  gradInput : CudaTensor - empty
  _type : torch.CudaTensor
  hyDesc : cdata<struct cudnnTensorStruct *[?]>: 0x41057f10
  wDesc : cdata<struct cudnnFilterStruct *[?]>: 0x403d6708
  seqLength : 13
  cyDesc : cdata<struct cudnnTensorStruct *[?]>: 0x40d89e28
  bidirectional : CUDNN_BIDIRECTIONAL
  numDirections : 2
  gradCellInput : CudaTensor - empty
  miniBatch : 20
  mode : CUDNN_RNN_RE

   13
   20
 3520
[torch.LongStorage of size 3]



In [81]:
print(Network.model:get(2):get(1):get(6))
d16 = Network.model:get(2):get(1):get(6):forward(d15)
print(d16:size())

   13
   20
 3520
[torch.LongStorage of size 3]



In [91]:
print(Network.model:get(2):get(1):get(7))
d17 = Network.model:get(2):get(1):get(7):forward(d16)
print(d17:size())

nn.View(-1, 2, 1760)
{
  _type : torch.CudaTensor
  output : CudaTensor - size: 13x20x2x1760
  gradInput : CudaTensor - empty
  size : LongStorage - size: 3
  numElements : 3520
  numInputDims : 2
}
   13
   20
    2
 1760
[torch.LongStorage of size 4]

